In [ ]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [ ]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

## פעולות מטריציוניות
### @ ו-np.matmul
האופרטור @ שקול ל-np.matmul עבור מכפלות מטריציות. שניהם תומכים בבאצ'ינג (batch/broadcast) במימדים גבוהים.
שימו לב כי גם האופרטור וגם הפונקציה **לא** מבצעים כפל איבר-איבר; לשם כך משתמשים ב-*.

### np.dot
עבור שני וקטורים – מחזיר מכפלה סקלרית.
עבור מטריצות דו-ממדיות – שקול למכפלת מטריצות.
במימדים גבוהים הכללים שונים מעט (כפל לאורך הצירים האחרונים) ולכן נעדיף @/matmul לקוד ברור.

![Alt text](figs/0N7TO.png)



נדגים עבור שתי מטריצות ממימד 2:

In [ ]:
import numpy as np
A = np.array([[2., 1.],
              [0., 3.]])
B = np.array([[1., -1.],
              [4.,  2.]])

AB1 = A @ B            # matrix product via @
AB2 = np.matmul(A, B)  # same
AB3 = np.dot(A, B)     # same for 2D
print("AB1:")
print(AB1)
print("AB2:")
print(AB2)
print("AB3:")
print(AB3)


AB1:
[[ 6.  0.]
 [12.  6.]]
AB2:
[[ 6.  0.]
 [12.  6.]]
AB3:
[[ 6.  0.]
 [12.  6.]]


In [1]:
import json
from jupyterquiz import display_quiz

example = \
'''
[{
  "question": "נתונות המטריצות:<br><br>A = np.array([[1,2],[3,4]])<br><br>B = np.array([[5,6],[7,8]])<br><br>מה ההבדל בין np.dot(A,B) לבין np.matmul(A,B)?",
  "type": "many_choice",
  "answers": [
    {
      "answer": "אין הבדל: שתיהן מחזירות את [[19,22],[43,50]]",
      "correct": true,
      "feedback": "נכון! עבור מטריצות דו־ממדיות np.dot ו־np.matmul זהות"
    },
    {
      "answer": "np.dot מבצע מכפלה איבר־לאיבר ו־np.matmul מכפלה מטריציונית",
      "correct": false,
      "feedback": "לא נכון — מכפלה איבר־לאיבר נעשית עם * ולא עם dot"
    },
    {
      "answer": "np.dot מחזיר סקלר ו־np.matmul מחזיר מטריצה",
      "correct": false,
      "feedback": "זה נכון רק כשעובדים עם וקטורים חד־ממדיים, לא עם מטריצות 2D"
    },
    {
      "answer": "np.matmul עובד רק על מטריצות ריבועיות ואילו np.dot לא",
      "correct": false,
      "feedback": "לא נכון — np.matmul עובד גם על מטריצות מלבניות כל עוד הממדים תואמים"
    }
  ]
}]
'''
myquiz = json.loads(example)
display_quiz(myquiz)


<IPython.core.display.Javascript object>

### transpose / .T
טרנספוז של מטריצה 2D מחליף שורות בעמודות.
![Alt text](https://datascienceparichay.com/wp-content/uploads/2021/07/transpose-a-numpy-array.png)

In [ ]:
A_T = A.T
B_T = B.T

AB1, AB2, np.allclose(AB1, AB2), np.allclose(AB1, AB3), A_T, B_T




### Transpose למימדים גבוהים יותר
במערכים בעלי יותר משני ממדים (tensors), `transpose` מחליפה **סדר צירים** כללי.  
ברירת המחדל היא היפוך של סדר הצירים (axis reversal), אבל ניתן לשלוט בסדר הרצוי דרך ארגומנט `axes`.

- לדוגמה: מערך 3D מצורה `(time, x, y)`, לאחר  אם נעשה `np.transpose(A)`, יתקבל `(y, x, time)`.  
- שימושי במיוחד במעבר בין תיאורים שונים של נתונים (למשל, בין **row-major** ל-**column-major**, או סידור ערוצים בנתוני תמונה).

נמחיש זאת בדוגמא הבאה:

In [ ]:
# 3D tensor of shape (2, 2, 3)
T = np.arange(12).reshape(2, 2, 3)

T_default = np.transpose(T)           # reverses axes -> shape (3, 2, 2)

T.shape, T_default.shape

((2, 2, 3), (3, 2, 2))

![Alt text](https://ajcr.net/images/stride-guide/3d_array_c_transpose.png)


כמו כן, ניתן לסמן צירים ולבחור את הסדר המתקבל:

In [ ]:
# 3D tensor of shape (2, 3, 4)
T = np.arange(24).reshape(2, 3, 4)

T_default = np.transpose(T)           # reverses axes -> shape (4, 3, 2)
T_custom  = np.transpose(T, axes=(1, 0, 2))  # custom order -> shape (3, 2, 4)

T.shape, T_default.shape, T_custom.shape

In [7]:
import json
from jupyterquiz import display_quiz

example = \
'''
[{
  "question": "נתון מערך בגודל (2,3,4). מה תהיה הצורה החדשה אם נבצע <br><br>transpose(1,0,2)<br><br>עם סדר צירים ?",
  "type": "many_choice",
  "answers": [
    {
      "answer": "(3,2,4)",
      "correct": true,
      "feedback": "נכון! החלפנו בין הציר הראשון והשני, השלישי נשאר במקום"
    },
    {
      "answer": "(2,4,3)",
      "correct": false,
      "feedback": "לא נכון — זה היה קורה אם היינו מחליפים בין הציר השני לשלישי"
    },
    {
      "answer": "(4,3,2)",
      "correct": false,
      "feedback": "לא נכון — זה קורה אם עושים A.T כלומר היפוך כל הצירים"
    },
    {
      "answer": "(2,3,4)",
      "correct": false,
      "feedback": "לא נכון — זה היה מתקבל אם לא היינו משנים את סדר הצירים כלל"
    }
  ]
}]
'''
myquiz = json.loads(example)
display_quiz(myquiz)


<IPython.core.display.Javascript object>

### פעולות מטריציוניות עם Batch ו־Broadcasting

ב־NumPy ניתן לבצע מכפלות מטריציוניות לא רק בין מטריצות בודדות, אלא גם בין *אוספים* של מטריצות 
(batch) בצורה וקטורית.

נראה שתי דוגמאות:
1. **מכפלת מטריצה–וקטור עבור batch של מטריצות**:  
   כל מטריצה ב־batch מוכפלת באותו וקטור.
2. **מכפלת מטריצה–מטריצה עבור batch של מטריצות**:  
   כל זוג מטריצות תואם ב־batch מוכפל אחת בשנייה.

In [ ]:
# Batched matrix-vector multiply: shape (batch, n, n) @ (n,)
batch = 5
M = rng.normal(size=(batch, 3, 3))
v = rng.normal(size=3)

# Broadcasting: each 3x3 in M multiplies the same vector v -> (batch, 3)
Mv = M @ v

במקרה זה התקבלה תוצאה במימדים `(batch, 3)`, כלומר 5 וקטורים שונים באורך 3 —
כל אחד תוצאה של כפל מטריצה מה־batch עם אותו וקטור $v$.

In [ ]:
N = rng.normal(size=(batch, 3, 3))  # another batch of 5 random 3x3 matrices

# Batched matrix-matrix: (batch, n, n) @ (batch, n, n) -> (batch, n, n)
MM = M @ N
MM.shape

כאן התוצאה במימדים `(batch, 3, 3)` — כלומר 5 מטריצות חדשות, שכל אחת מהן תוצאה 
של כפל זוג המטריצות המתאים מתוך $M$ ו־$N$.

In [6]:
import json
from jupyterquiz import display_quiz

example = \
'''
[{
  "question": "נתון הקוד הבא:<br><br>\
A = np.ones((5, 3, 3))<br>\
v = np.ones(3)<br><br>\
מה תהיה הצורה של A @ v ?",
  "type": "many_choice",
  "answers": [
    {
      "answer": "הצורה (5,3)",
      "correct": true,
      "feedback": "נכון! כל אחת מ־5 המטריצות בגודל 3x3 מוכפלת באותו וקטור באורך 3"
    },
    {
      "answer": "הצורה (3,3)",
      "correct": false,
      "feedback": "לא נכון — זה היה נכון אילו A הייתה מטריצה אחת בלבד"
    },
    {
      "answer": "הצורה (5,3,3)",
      "correct": false,
      "feedback": "לא נכון — זו הייתה התוצאה אילו היינו עושים broadcasting עם כפל איבר־לאיבר"
    },
    {
      "answer": "הצורה (,5)",
      "correct": false,
      "feedback": "לא נכון — זה היה מתקבל אם כל מטריצה הייתה מוכפלת בסקלר"
    }
  ]
}]
'''
myquiz = json.loads(example)
display_quiz(myquiz)


<IPython.core.display.Javascript object>